# LangGraph StateGraph로 챗봇 만들기

이 노트북에서는 **LangGraph**를 사용하여 **상태(State)를 관리하는 챗봇**을 만드는 방법을 배웁니다.

## LangGraph란?

**LangGraph**는 LangChain 팀이 만든 라이브러리로, **그래프 구조**로 AI 워크플로우를 만들 수 있게 해줍니다.

### 왜 LangGraph를 사용할까요?

```
┌────────────────────────────────────────────────────────────────────┐
│                LangChain vs LangGraph                             │
├────────────────────────────────────────────────────────────────────┤
│                                                                    │
│   LangChain (체인):                                                │
│   입력 → 처리1 → 처리2 → 처리3 → 출력                             │
│   (일직선으로 흐름)                                                │
│                                                                    │
│   LangGraph (그래프):                                              │
│                 ┌──→ 처리A ──┐                                     │
│   입력 → 분기점 ─┼──→ 처리B ──┼→ 합류점 → 출력                     │
│                 └──→ 처리C ──┘                                     │
│   (복잡한 흐름, 조건 분기, 반복 가능)                              │
│                                                                    │
└────────────────────────────────────────────────────────────────────┘
```

### StateGraph의 구성 요소

```
┌────────────────────────────────────────────────────────────────────┐
│                  StateGraph의 3가지 요소                           │
├────────────────────────────────────────────────────────────────────┤
│                                                                    │
│   1️⃣ State (상태):                                                │
│      • 그래프 전체에서 공유되는 데이터                              │
│      • 예: 대화 기록, 사용자 정보 등                               │
│                                                                    │
│   2️⃣ Node (노드):                                                 │
│      • 실제 작업을 수행하는 함수                                   │
│      • 예: LLM 호출, 검색, 계산 등                                 │
│                                                                    │
│   3️⃣ Edge (엣지):                                                 │
│      • 노드 간의 연결                                              │
│      • 어떤 순서로 실행될지 결정                                   │
│                                                                    │
└────────────────────────────────────────────────────────────────────┘
```

---

# 1. 환경 설정

In [ ]:
!pip install -q langchain langchain-ollama langgraph

In [ ]:
import subprocess
import time

!apt-get install -y zstd
!curl -fsSL https://ollama.com/install.sh | sh

subprocess.Popen(['ollama', 'serve'])
time.sleep(3)

!ollama pull llama3.2

# 2. State (상태) 정의하기

## TypedDict란?

**TypedDict**는 딕셔너리의 키와 값 타입을 미리 정의하는 방법입니다.

```
┌────────────────────────────────────────────────────────────────────┐
│                    TypedDict 쉬운 설명                             │
├────────────────────────────────────────────────────────────────────┤
│                                                                    │
│   일반 딕셔너리:                                                   │
│   data = {"name": "민혁", "age": 25}                              │
│   → 어떤 키가 있는지, 값 타입이 뭔지 모름                          │
│                                                                    │
│   TypedDict:                                                       │
│   class Person(TypedDict):                                         │
│       name: str                                                    │
│       age: int                                                     │
│   → "name은 문자열, age는 숫자"라고 명시                           │
│                                                                    │
└────────────────────────────────────────────────────────────────────┘
```

## add_messages란?

**add_messages**는 메시지를 **덮어쓰지 않고 추가**하는 방법입니다.

```
┌────────────────────────────────────────────────────────────────────┐
│                    add_messages의 역할                             │
├────────────────────────────────────────────────────────────────────┤
│                                                                    │
│   일반적인 딕셔너리 업데이트:                                       │
│   state = {"messages": ["안녕"]}                                   │
│   state = {"messages": ["반가워"]}  # 덮어씀!                      │
│   결과: ["반가워"] ❌                                              │
│                                                                    │
│   add_messages 사용:                                               │
│   state = {"messages": ["안녕"]}                                   │
│   state = add_messages(["반가워"])  # 추가!                        │
│   결과: ["안녕", "반가워"] ✅                                      │
│                                                                    │
└────────────────────────────────────────────────────────────────────┘
```

In [ ]:
from typing import Annotated, TypedDict
from langgraph.graph import add_messages

# 상태 정의
class State(TypedDict):
    """
    챗봇의 상태를 정의합니다.
    
    messages: 대화 기록 리스트
    - Annotated[list, add_messages]: 새 메시지를 추가하는 방식으로 업데이트
    """
    messages: Annotated[list, add_messages]

print("✅ State 정의 완료")
print("   - messages: 대화 기록을 저장하는 리스트")

# 3. Node (노드) 정의하기

노드는 실제 작업을 수행하는 함수입니다. 챗봇 노드는 LLM을 호출합니다.

In [ ]:
from langchain_ollama import ChatOllama

# LLM 모델
model = ChatOllama(model='llama3.2')

def chatbot(state: State):
    """
    챗봇 노드: LLM을 호출하여 응답을 생성합니다.
    
    입력: state (현재 상태, 대화 기록 포함)
    출력: {"messages": [응답]} (새 메시지를 추가)
    """
    # 현재까지의 대화 기록으로 LLM 호출
    answer = model.invoke(state['messages'])
    
    # 응답을 메시지 리스트에 추가
    return {'messages': [answer]}

print("✅ chatbot 노드 정의 완료")

# 4. Graph (그래프) 구성하기

노드와 엣지를 연결하여 그래프를 만듭니다.

```
┌────────────────────────────────────────────────────────────────────┐
│                    챗봇 그래프 구조                                │
├────────────────────────────────────────────────────────────────────┤
│                                                                    │
│          ┌─────────┐      ┌─────────┐      ┌─────────┐            │
│          │  START  │ ───▶ │ chatbot │ ───▶ │   END   │            │
│          └─────────┘      └─────────┘      └─────────┘            │
│                                                                    │
│   START: 그래프 시작점                                             │
│   chatbot: LLM 호출 노드                                          │
│   END: 그래프 종료점                                               │
│                                                                    │
└────────────────────────────────────────────────────────────────────┘
```

In [ ]:
from langgraph.graph import StateGraph, START, END

# 그래프 빌더 생성
builder = StateGraph(State)

# 노드 추가
# 첫 번째 인자: 노드 이름 (고유해야 함)
# 두 번째 인자: 실행할 함수
builder.add_node('chatbot', chatbot)

# 엣지 추가 (노드 간 연결)
builder.add_edge(START, 'chatbot')  # 시작 → 챗봇
builder.add_edge('chatbot', END)     # 챗봇 → 종료

# 그래프 컴파일
graph = builder.compile()

print("✅ 그래프 구성 완료")

# 5. 그래프 시각화

In [ ]:
# 그래프 구조를 Mermaid 형식으로 출력
print("=== 그래프 구조 (Mermaid) ===")
print(graph.get_graph().draw_mermaid())

# 6. 그래프 실행

## stream vs invoke

| 메서드 | 동작 | 사용 시나리오 |
|--------|------|---------------|
| **invoke** | 최종 결과만 반환 | 결과만 필요할 때 |
| **stream** | 단계별 결과 반환 | 진행 상황을 보고 싶을 때 |

In [ ]:
from langchain_core.messages import HumanMessage

# 입력 준비
input_message = {'messages': [HumanMessage(content='안녕하세요!')]}

print("=== 그래프 실행 (stream) ===")
print(f"입력: {input_message}\n")

# stream으로 실행 (단계별 출력)
for chunk in graph.stream(input_message):
    print(f"출력: {chunk}")

In [ ]:
# invoke로 실행 (최종 결과만)
result = graph.invoke(input_message)

print("=== 그래프 실행 (invoke) ===")
print(f"\n최종 응답: {result['messages'][-1].content}")

# 7. 여러 번 대화해보기

In [ ]:
# 여러 질문으로 테스트
test_messages = [
    "파이썬이란 무엇인가요?",
    "오늘 날씨가 어때요?",
    "간단한 파이썬 코드를 보여주세요.",
]

for msg in test_messages:
    print(f"\n{'='*50}")
    print(f"사용자: {msg}")
    
    result = graph.invoke({'messages': [HumanMessage(content=msg)]})
    
    print(f"\nAI: {result['messages'][-1].content[:200]}...")

---

## 정리: LangGraph StateGraph

### 핵심 구성요소

| 구성요소 | 역할 | 예시 |
|----------|------|------|
| **State** | 공유 데이터 정의 | `messages: list` |
| **Node** | 작업 수행 함수 | `chatbot()` |
| **Edge** | 노드 간 연결 | `START → chatbot → END` |

### 핵심 코드

```python
from langgraph.graph import StateGraph, START, END, add_messages

# 1. 상태 정의
class State(TypedDict):
    messages: Annotated[list, add_messages]

# 2. 노드 함수 정의
def chatbot(state: State):
    answer = model.invoke(state['messages'])
    return {'messages': [answer]}

# 3. 그래프 구성
builder = StateGraph(State)
builder.add_node('chatbot', chatbot)
builder.add_edge(START, 'chatbot')
builder.add_edge('chatbot', END)

# 4. 컴파일 및 실행
graph = builder.compile()
result = graph.invoke({'messages': [...]})
```

### LangGraph의 장점

| 장점 | 설명 |
|------|------|
| **상태 관리** | State로 데이터 흐름 명확히 관리 |
| **확장성** | 노드 추가로 기능 확장 용이 |
| **시각화** | 그래프 구조 시각적 확인 가능 |
| **유연성** | 조건 분기, 반복 등 복잡한 흐름 구현 |

## 코드 변경점 (OpenAI → Ollama)

```python
# 원본
from langchain_openai import ChatOpenAI
model = ChatOpenAI(model='gpt-4o-mini')

# 변경
from langchain_ollama import ChatOllama
model = ChatOllama(model='llama3.2')
```

## 다음 단계

**MemorySaver**를 사용하여 대화 기록을 **영구 저장**하는 방법을 배웁니다. (07-10번 노트북)